In [1]:
import yfinance as yh
import pandas as pd
import matplotlib.pyplot as plt
import polars as pl
from datetime import datetime
import pytz
from datetime import datetime, timedelta
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import utils as ut
from sklearn.ensemble import RandomForestClassifier
import warnings
import pickle
warnings.filterwarnings("ignore")

In [2]:
symbol_train = [
     "AAPL", "MSFT", "GOOG", "AMZN", "META", "TSLA",
    "CRM", "AMD", "BABA", "INTC", "PYPL", "TTD", "EA", 
    "ZG", "MTCH", "YELP", "ADBE", "ORCL", "SAP", "CSCO", "UBER", "LYFT", "ZM",
    "NFLX", "SNAP", "SHOP", "TWLO", "SQ", "SPOT", "NET", "ROKU",
    "DOCU", "PANW", "CRWD", "SNOW", "ASML"
]

In [4]:
symbol = "NVDA"
info = yh.Ticker(symbol)
data = pl.DataFrame(info.history(period="5Y")[['Open', 'High', 'Low', 'Close', 'Volume']].reset_index())
data = data.with_columns(pl.lit(symbol).alias("Symbol"))

data = data.with_columns(
    (pl.col("Close") / pl.col("Close").shift(1) - 1).alias("daily_return")
)


In [15]:
symbol = "NVDA"
info = yh.Ticker(symbol)
data = pl.DataFrame(info.history(period="5Y")[['Open', 'High', 'Low', 'Close', 'Volume']].reset_index())
data = data.with_columns(pl.lit(symbol).alias("Symbol"))

data = data.with_columns(
    (pl.col("Close") / pl.col("Close").shift(1) - 1).alias("daily_return")
)

current_date = datetime.now(pytz.timezone('America/New_York'))
last_year_date = current_date - timedelta(days=365)

data = data.with_columns(
        pl.when(pl.col('daily_return').shift(-1)>0).then(1).otherwise(0).alias("target_1"),
        pl.when(pl.col('daily_return').shift(-1)>0.05).then(2).when(pl.col('daily_return').shift(-1)<-0.05).then(1).otherwise(0).alias("target_2"),
        pl.when(pl.col('daily_return').shift(-5)>0.05).then(2).when(pl.col('daily_return').shift(-5)<-0.05).then(1).otherwise(0).alias("target_3"),
        pl.when(pl.col('daily_return').shift(-3)>0.05).then(2).when(pl.col('daily_return').shift(-3)<-0.05).then(1).otherwise(0).alias("target_4")
    )

data = data.with_columns(
        pl.when(pl.col("Date") >= pl.lit(last_year_date).cast(pl.Datetime).dt.convert_time_zone('America/New_York'))
        .then(pl.lit("test"))
        .otherwise(pl.lit("train"))
        .alias("period")
    )



data = ut.bollinger_band_comb(data)
data = ut.rsi_comb(data)
data = ut.stochastic_oscillator_comb(data)
data = ut.atr_comb(data)
data = ut.obv_comb(data)
data = ut.williams_comb(data)
data = ut.adx_comb(data)
data = ut.aroon_comb(data)
data = ut.keltner_comb(data)
data = ut.cmf_comb(data)

#start_date = data[30, "Date"]
#data = data.filter(pl.col("Date") >= start_date)

for symbol in symbol_train:
    info = yh.Ticker(symbol)
    df = pl.DataFrame(info.history(period="5Y")[['Open', 'High', 'Low', 'Close', 'Volume']].reset_index())
    df = df.with_columns(pl.lit(symbol).alias("Symbol"))

    df = df.with_columns(
    (pl.col("Close") / pl.col("Close").shift(1) - 1).alias("daily_return")
)

    df = df.with_columns(
        pl.when(pl.col('daily_return').shift(-1)>0).then(1).otherwise(0).alias("target_1"),
        pl.when(pl.col('daily_return').shift(-1)>0.05).then(2).when(pl.col('daily_return').shift(-1)<-0.05).then(1).otherwise(0).alias("target_2"),
        pl.when(pl.col('daily_return').shift(-5)>0.05).then(2).when(pl.col('daily_return').shift(-5)<-0.05).then(1).otherwise(0).alias("target_3"),
        pl.when(pl.col('daily_return').shift(-3)>0.05).then(2).when(pl.col('daily_return').shift(-3)<-0.05).then(1).otherwise(0).alias("target_4")
    )

    df = df.with_columns(
        pl.when(pl.col("Date") >= pl.lit(last_year_date).cast(pl.Datetime).dt.convert_time_zone('America/New_York'))
        .then(pl.lit("test"))
        .otherwise(pl.lit("train"))
        .alias("period")
    )

    df = ut.bollinger_band_comb(df)
    df = ut.rsi_comb(df)
    df = ut.stochastic_oscillator_comb(df)
    df = ut.atr_comb(df)
    df = ut.obv_comb(df)
    df = ut.williams_comb(df)
    df = ut.adx_comb(df)
    df = ut.aroon_comb(df)
    df = ut.keltner_comb(df)
    df = ut.cmf_comb(df)

    #start_date = df[30, "Date"]

    # Filter the DataFrame to keep only rows with dates greater than or equal to the 31st day
    #df = df.filter(pl.col("Date") >= start_date)

    data = pl.concat([data, df])
    print(symbol, ' OK')

AAPL  OK
MSFT  OK
GOOG  OK
AMZN  OK
META  OK
TSLA  OK
CRM  OK
AMD  OK
BABA  OK
INTC  OK
PYPL  OK
TTD  OK
EA  OK
ZG  OK
MTCH  OK
YELP  OK
ADBE  OK
ORCL  OK
SAP  OK
CSCO  OK
UBER  OK
LYFT  OK
ZM  OK
NFLX  OK
SNAP  OK
SHOP  OK
TWLO  OK
SQ  OK
SPOT  OK
NET  OK
ROKU  OK
DOCU  OK
PANW  OK
CRWD  OK
SNOW  OK
ASML  OK


In [16]:
data

Date,Open,High,Low,Close,Volume,Symbol,daily_return,target_1,target_2,target_3,target_4,period,signal_bollinger_12_1,signal_bollinger_12_2,signal_bollinger_20_1,signal_bollinger_20_2,RSI_14_30_70,signal_rsi_14_30_70,RSI_14_20_80,signal_rsi_14_20_80,RSI_8_30_70,signal_rsi_8_30_70,RSI_8_20_80,signal_rsi_8_20_80,Stochastic_K_14_3,Stochastic_D_14_3,Stochastic_K_14_5,Stochastic_D_14_5,Stochastic_K_8_3,Stochastic_D_8_3,Stochastic_K_8_5,Stochastic_D_8_5,signal_atr_14_1,signal_atr_14_1.5,signal_atr_8_1,signal_atr_8_1.5,signal_atr_30_1,signal_atr_30_1.5,OBV_trend,signal_obv_1,signal_obv_3,signal_obv_4,Williams_R_14,Williams_R_8,Williams_R_30,adx_14,adx_8,adx_30,Aroon_Oscillator_14,Aroon_Oscillator_8,Aroon_Oscillator_30,Keltner_20_1.5,Keltner_20_2,Keltner_20_2.5,Keltner_14_1.5,Keltner_14_2,Keltner_14_2.5,Keltner_8_1.5,Keltner_8_2,Keltner_8_2.5,CMF_20,CMF_14,CMF_8
"datetime[ns, America/New_York]",f64,f64,f64,f64,i64,str,f64,i32,i32,i32,i32,str,i32,i32,i32,i32,f64,i32,f64,i32,f64,i32,f64,i32,f64,f64,f64,f64,f64,f64,f64,f64,i32,i32,i32,i32,i32,i32,i32,i32,i32,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
2019-11-25 00:00:00 EST,5.374856,5.508456,5.361919,5.50348,506476000,"""NVDA""",null,0,0,0,0,"""train""",0,0,0,0,null,0,null,0,null,0,null,0,96.604341,null,96.604341,null,96.604341,null,96.604341,null,0,0,0,0,0,0,-1,0,0,0,-3.395659,-3.395659,-3.395659,0.0,0.0,0.0,0.0,0.0,0.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,0.932087,0.932087,0.932087
2019-11-26 00:00:00 EST,5.485817,5.490544,5.392272,5.398741,385244000,"""NVDA""",-0.019032,1,0,0,0,"""train""",0,0,0,0,null,0,null,0,null,0,null,0,25.127935,null,25.127935,null,25.127935,null,25.127935,null,0,0,0,0,0,0,-1,-1,0,0,-74.872065,-74.872065,-74.872065,3.474787,3.474787,3.474787,0.0,0.0,0.0,36.801723,40.101292,42.081034,36.956089,40.217066,42.173653,37.35744,40.51808,42.414464,0.154256,0.154256,0.154256
2019-11-27 00:00:00 EST,5.444303,5.478661,5.407206,5.433597,215784000,"""NVDA""",0.006456,0,0,0,0,"""train""",0,0,0,0,null,0,null,0,null,0,null,0,48.914246,56.882174,48.914246,null,48.914246,56.882174,48.914246,null,0,0,0,0,0,0,1,0,0,0,-51.085754,-51.085754,-51.085754,5.257855,5.257855,5.257855,0.0,0.0,0.0,46.889809,47.667357,48.133886,47.040331,47.780249,48.224199,47.419843,48.064882,48.451906,0.073284,0.073284,0.073284
2019-11-29 00:00:00 EST,5.402724,5.431107,5.372349,5.396251,141556000,"""NVDA""",-0.006873,0,0,0,0,"""train""",0,0,0,0,null,0,null,0,null,0,null,0,23.42868,32.490287,23.42868,null,23.42868,32.490287,23.42868,null,0,0,0,0,0,0,-1,0,0,0,-76.57132,-76.57132,-76.57132,7.995159,7.995159,7.995159,0.0,0.0,0.0,38.318103,41.238577,42.990862,38.582802,41.437102,43.149681,39.255601,41.9417,43.55336,0.04385,0.04385,0.04385
2019-12-02 00:00:00 EST,5.389279,5.402724,5.194582,5.209769,372036000,"""NVDA""",-0.034558,0,0,0,0,"""train""",0,0,0,0,null,0,null,0,null,0,null,0,4.838598,25.727175,4.838598,39.78276,4.838598,25.727175,4.838598,39.78276,-1,-1,-1,-1,-1,-1,-1,-1,-1,0,-95.161402,-95.161402,-95.161402,14.382851,14.382851,14.382851,28.571429,50.0,13.333333,2.642962,14.482222,21.585777,3.397447,15.048085,22.038468,5.379731,16.534798,23.227838,-0.162219,-0.162219,-0.162219
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2024-11-18 00:00:00 EST,651.070007,668.859985,648.0,665.22998,2012600,"""ASML""",0.010021,0,0,0,0,"""test""",0,0,0,0,34.487487,0,34.487487,0,52.060728,0,52.060728,0,26.921844,31.888231,26.921844,29.029475,26.921844,33.465766,26.921844,41.575347,0,0,0,0,0,0,1,1,0,0,-73.078156,-73.078156,-92.294618,25.8482,18.296094,16.702068,14.285714,25.0,80.0,-56.22773,-29.670797,-13.736638,-18.119727,-1.089795,9.128164,21.237422,28.428067,32.742453,-0.168425,-0.211265,0.035719
2024-11-19 00:00:00 EST,662.820007,671.409973,653.469971,662.159973,1640700,"""ASML""",-0.004615,0,0,0,0,"""test""",0,0,-1,0,41.830655,0,41.830655,0,40.533383,0,40.533383,0,22.124958,18.452354,22.